# Домашнее задание №4


- Необходимо будет перебрать $N$ моделей $(N \geq 2)$ матричной факторизации и перебрать у них $K$ гиперпараметров $(K \geq 2)$ 
    - Для перебора гиперпараметров можно использовать [`Optuna`](https://github.com/optuna/optuna), [`Hyperopt`](https://github.com/hyperopt/hyperopt)
- Воспользоваться методом приближенного поиска соседей для выдачи рекомендаций. 
    - Можно использовать любые удобные: [`Annoy`](https://github.com/spotify/annoy), [`nmslib`](https://github.com/nmslib/nmslib) и.т.д
- Добавить 3 "аватаров" (искусственных пользователей) и посмотреть рекомендации итоговой модели на них. Объяснить почему добавили именно таких пользователей.
- Придумать как можно обработать рекомендации для холодных пользователей. 

## Импорты

In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"  # For implicit ALS
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import typing as tp
from pathlib import Path

import dill
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.lmf import LogisticMatrixFactorization
from lightfm import LightFM
from rectools import Columns
from rectools.dataset import Dataset
from rectools.metrics import MAP, Precision, Recall, calc_metrics
from rectools.models import (
    ImplicitALSWrapperModel,
    LightFMWrapperModel,
)
from tqdm import tqdm

In [5]:
PATH = Path("/content/drive/MyDrive/ML_Eng/kion_train")

## Подготовка данных

In [6]:
users = pd.read_csv(PATH / 'users.csv')
items = pd.read_csv(PATH / 'items.csv')
interactions = pd.read_csv(PATH / 'interactions.csv')

In [7]:
Columns.Datetime = 'last_watch_dt'
     

interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')
max_date = interactions[Columns.Datetime].max()
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)
     

# Разделяем на train и test
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()
     

train.drop(train.query("total_dur < 300").index, inplace=True)
cold_users = set(test[Columns.User]) - set(train[Columns.User])

# Отбрасываем холодных пользователей
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

In [38]:
dataset = Dataset.construct(interactions_df=train)
metric = MAP(k = 10)

## Применение Optuna для подбора гиперпараметров

## ALS Model

In [39]:
def als_optuna_objective(trial):

    factors = trial.suggest_int('factors', 32, 64)
    regularization = trial.suggest_float('regularization', 0.001, 0.1, log=True)
    iterations = trial.suggest_int('iterations', 5, 20)

    model = ImplicitALSWrapperModel(
        model=AlternatingLeastSquares(
            factors=factors,
            regularization=regularization,
            iterations = iterations,
            random_state=42, 
            num_threads=16,
            use_gpu=True
        ))
    model.fit(dataset)
    recos = model.recommend(
        users=test[Columns.User].unique(),
        dataset=dataset,
        k=10,
        filter_viewed=True,
    )

    map10 = metric.calc_per_user(recos, test)
    return map10.mean()


In [40]:
# перебор параметров с помощью optuna
study = optuna.create_study(directions = ['maximize'])
study.optimize(als_optuna_objective, n_trials=20)

[I 2023-04-08 23:49:03,395] A new study created in memory with name: no-name-6cb12ffd-fd39-454a-a12a-550490545496
[I 2023-04-08 23:50:30,582] Trial 0 finished with value: 0.02471826347911964 and parameters: {'factors': 38, 'regularization': 0.03170807995620261, 'iterations': 18}. Best is trial 0 with value: 0.02471826347911964.
[I 2023-04-08 23:51:55,420] Trial 1 finished with value: 0.024644943387009413 and parameters: {'factors': 52, 'regularization': 0.008551199544889988, 'iterations': 13}. Best is trial 0 with value: 0.02471826347911964.
[I 2023-04-08 23:53:13,971] Trial 2 finished with value: 0.023833529527120388 and parameters: {'factors': 45, 'regularization': 0.08850469342116962, 'iterations': 7}. Best is trial 0 with value: 0.02471826347911964.
[I 2023-04-08 23:54:32,616] Trial 3 finished with value: 0.023988272465690175 and parameters: {'factors': 50, 'regularization': 0.09646012068473976, 'iterations': 8}. Best is trial 0 with value: 0.02471826347911964.
[I 2023-04-08 23:55:

In [41]:
# лучшие параметры
print(f'Лучшее значение MAP@10: {study.best_value}')
print(f'Лучшие параметры: {study.best_params}')

Лучшее значение MAP@10: 0.02762808458280813
Лучшие параметры: {'factors': 32, 'regularization': 0.006450307654944045, 'iterations': 18}


## LightFM Model

In [42]:
# введем функцию по подбору оптимальных гиперпараметров
def lightfm_optuna_objective(trial):
    
    no_components = trial.suggest_int('no_components', 32, 256)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05, log=True)
    loss = trial.suggest_categorical("loss", ['bpr', 'warp', 'logistic'])
    
        
    model = LightFMWrapperModel(
                                 model = LightFM(
                                 no_components = no_components,
                                 loss = loss,
                                 rho = 0.2,
                                 epsilon = 1e-5,
                                 learning_rate = learning_rate,
                                 random_state = 42,
                                 ),
                                  epochs=1,
                                  num_threads=16
              )

    
    model.fit(dataset)
    recos = model.recommend(
        users=test[Columns.User].unique(),
        dataset=dataset,
        k=10,
        filter_viewed=True,
    )

    map10 = metric.calc_per_user(recos, test)
    return map10.mean()

In [43]:
# перебор параметров с помощью optuna
study = optuna.create_study(directions = ['maximize'])
study.optimize(lightfm_optuna_objective, n_trials=20)

[I 2023-04-09 00:17:07,726] A new study created in memory with name: no-name-78836a68-6c38-42e7-b022-a6a053b8c757
[I 2023-04-09 00:21:01,962] Trial 0 finished with value: 0.07471844266608645 and parameters: {'no_components': 244, 'learning_rate': 0.0439253650987498, 'loss': 'logistic'}. Best is trial 0 with value: 0.07471844266608645.
[I 2023-04-09 00:22:54,640] Trial 1 finished with value: 0.05862527442768599 and parameters: {'no_components': 78, 'learning_rate': 0.040378164775394444, 'loss': 'bpr'}. Best is trial 0 with value: 0.07471844266608645.
[I 2023-04-09 00:25:27,936] Trial 2 finished with value: 0.07471815113322634 and parameters: {'no_components': 132, 'learning_rate': 0.014500343768065532, 'loss': 'logistic'}. Best is trial 0 with value: 0.07471844266608645.
[I 2023-04-09 00:27:29,276] Trial 3 finished with value: 0.07471809159729888 and parameters: {'no_components': 80, 'learning_rate': 0.04938599783890662, 'loss': 'logistic'}. Best is trial 0 with value: 0.074718442666086

In [44]:
# лучшие параметры
print(f'Лучшее значение MAP@10: {study.best_value}')
print(f'Лучшие параметры: {study.best_params}')

Лучшее значение MAP@10: 0.07880836726631638
Лучшие параметры: {'no_components': 246, 'learning_rate': 0.020619039200214098, 'loss': 'warp'}


## Обучение лучшей модели

## Подготовка фич

### User features

In [8]:
users.fillna('Unknown', inplace=True)
users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()

In [9]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


### Item features

In [10]:
items = pd.read_csv(PATH / 'items.csv')

In [11]:
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

In [12]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [13]:
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

In [14]:
countries_feature = items.reindex(columns=[Columns.Item, "countries"])
countries_feature.columns = ["id", "value"]
countries_feature["feature"] = "countries"

In [15]:
item_features = pd.concat((genre_feature, content_feature, countries_feature))

### LightFM Model

In [16]:
dataset = Dataset.construct(
    interactions_df=interactions,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "countries"],
)

TEST_USERS = test[Columns.User].unique()

In [17]:
# Лучшие параметры: {'no_components': 246, 'learning_rate': 0.020619039200214098, 'loss': 'warp'}
model = LightFMWrapperModel(
    LightFM(
        no_components=246,
        learning_rate=0.020619, 
        loss='warp',
        rho=0.2,
        epsilon = 1e-5,
        user_alpha=0,
        item_alpha=0,
        random_state=42,
    ),
    epochs=1,
    num_threads=16,
)
model.fit(dataset)

In [18]:
user_embeddings, item_embeddings = model.get_vectors(dataset)

user_id_map = dataset.user_id_map.to_internal
item_id_map = dataset.item_id_map.to_internal
item_id_inv_map = {idx:item_id for item_id, idx in item_id_map.items()}

In [19]:
output = user_embeddings[9].dot(item_embeddings.T)
recs = (-output).argsort()[:10]
recs = [item_id_inv_map[item_id] for item_id in recs]
recs

[10440, 13865, 15297, 9728, 2657, 9996, 4151, 142, 8636, 4740]

In [32]:
# сохраним модель для онлайн выдачи
with open('lightfm_model.dill', 'wb') as f:
    dill.dump(model, f)

In [33]:
# проверим, что модель выдает рекомендации для user_id = 10347 
model.recommend(pd.DataFrame([10347])[0], dataset, k=10, filter_viewed = True)


,user_id,item_id,score,rank
0,10347,10440,5.116018,1
1,10347,9728,4.996714,2
2,10347,15297,4.906498,3
3,10347,13865,4.828973,4
4,10347,3734,4.732056,5
5,10347,4151,4.707777,6
6,10347,2657,4.463011,7
7,10347,4880,4.427715,8
8,10347,1844,4.357654,9
9,10347,14741,4.317762,10


## Приближенный поиск соседей

In [21]:
!pip install nmslib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import nmslib

In [23]:
def augment_inner_product(factors):
    normed_factors = np.linalg.norm(factors, axis=1)
    max_norm = normed_factors.max()
    
    extra_dim = np.sqrt(max_norm ** 2 - normed_factors ** 2).reshape(-1, 1)
    augmented_factors = np.append(factors, extra_dim, axis=1)
    return max_norm, augmented_factors

In [24]:
max_norm, augmented_item_embeddings = augment_inner_product(item_embeddings)
augmented_item_embeddings.shape

(15706, 249)

In [25]:
extra_zero = np.zeros((user_embeddings.shape[0], 1))
augmented_user_embeddings = np.append(user_embeddings, extra_zero, axis=1)
augmented_user_embeddings.shape


(962179, 249)

In [26]:
M = 48
efC = 128
efS = 128
K = 10
num_threads = 4
space_name='negdotprod'

In [27]:
# инициализирем nmslib
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(augmented_item_embeddings) 

# создаем индекс
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params)

In [28]:
# задаем параметры для поиска
query_time_params = {'efSearch': efS}
index.setQueryTimeParams(query_time_params)

In [29]:
# получаем маппинги
user_id_map = dataset.user_id_map.to_internal
item_id_map = dataset.item_id_map.to_internal
item_id_inv_map = {idx:item_id for item_id, idx in item_id_map.items()}

In [30]:
test_users = [user_id_map[user] for user in TEST_USERS[:5]]
query_matrix = augmented_user_embeddings[test_users, :]
nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)

In [31]:
recs = [[item_id_inv_map[item] for item in user_nbrs[0]] for user_nbrs in nbrs]
recs[:5]


[[10440, 13865, 15297, 9728, 2657, 9996, 4151, 142, 8636, 4740],
 [9728, 10440, 13865, 15297, 3734, 4151, 4880, 2657, 6809, 142],
 [9728, 10440, 3734, 15297, 4151, 13865, 4880, 1844, 7571, 11237],
 [13865, 10440, 9728, 3734, 2657, 15297, 4880, 4151, 11237, 1819],
 [15297, 13865, 7571, 3182, 10440, 9728, 741, 4151, 16270, 7310]]

## Тестовые юзеры – Аватары

In [16]:
# сгенирируем подборку фильмов, которые смотрели аватары
ussr_fan = items[(items.countries.isna() == False) & (items.countries.str.contains('СССР'))][:5]
comedy_fan = items[(items.genres.isna() == False) & (items.genres.str.contains('комедии'))][:5]
magic_fan = items[(items.age_rating <= 16.0) & (items.keywords.str.contains('волшеб|чар|маг', case=False))][:5]

Первый юзер – мужчина средних лет, предпочитающий смотреть фильмы, которые выпустили в СССР

In [17]:
users = users.append({
    Columns.User: 10000001,
    'age': 'age_45_54',
    'income': 'income_40_60',
    'sex': 'M',
    'kids_flg': 0
}, ignore_index = True
)

In [18]:
ussr_fan = items.loc[items['item_id'].isin(ussr_fan.item_id), 
          ['item_id', 'title', 'content_type', 'countries', 'genre']]

In [19]:
ussr_fan

,item_id,title,content_type,countries,genre
4,16268,Все решает мгновение,film,СССР,"[драмы, спорт, советские, мелодрамы]"
6,1468,Марья-искусница,film,СССР,"[фильмы, сказки, приключения, советские, семей..."
13,16429,Ярослав Мудрый,film,СССР,"[для детей, драмы, исторические, советские, во..."
14,6181,"Первая встреча, последняя встреча",film,СССР,"[драмы, советские, комедии]"
29,6318,Днепровский ветер. Чары-камыши,film,СССР,"[драмы, русские, короткометражные]"


In [20]:
first_avatar = pd.DataFrame({
    'user_id': np.full(5, fill_value=10000001),
    'item_id': ussr_fan.item_id,
    'last_watch_dt': np.full(5, fill_value='2021-03-21'),
    'total_dur': np.full(5, fill_value=np.nan),
    'watched_pct': [80.0, 9.0, 50.0, 90.0, 8.0],
    'weight': [3, 1, 3, 3, 1]
    }
)

first_avatar[Columns.Datetime] = pd.to_datetime(first_avatar[Columns.Datetime], format='%Y-%m-%d')

Второй юзер – молодая девушка, предпочитающая смотреть комедии

In [21]:
users = users.append({
    Columns.User: 10000002,
    'age': 'age_25_34',
    'income': 'income_40_60',
    'sex': 'Ж',
    'kids_flg': 0
}, ignore_index = True
)

In [22]:
comedy_fan = items.loc[items['item_id'].isin(comedy_fan.item_id), 
          ['item_id', 'title', 'content_type', 'countries', 'genre']]

In [23]:
comedy_fan

,item_id,title,content_type,countries,genre
1,2508,Голые перцы,film,США,"[зарубежные, приключения, комедии]"
2,10716,Тактическая сила,film,Канада,"[криминал, зарубежные, триллеры, боевики, коме..."
14,6181,"Первая встреча, последняя встреча",film,СССР,"[драмы, советские, комедии]"
15,15076,Бладфест,film,США,"[зарубежные, ужасы, комедии]"
23,13109,Новый парень моей мамы,film,Германия,"[мелодрамы, зарубежные, криминал, комедии]"


In [24]:
second_avatar = pd.DataFrame({
    'user_id': np.full(5, fill_value=10000002),
    'item_id': comedy_fan.item_id,
    'last_watch_dt': np.full(5, fill_value='2021-05-25'),
    'total_dur': np.full(5, fill_value=np.nan),
    'watched_pct': [100.0, 5.0, 30.0, 5.0, 90.0],
    'weight': [3, 1, 3, 1, 3]
    }
)

second_avatar[Columns.Datetime] = pd.to_datetime(second_avatar[Columns.Datetime], format='%Y-%m-%d')

Третий юзер – молодая девушка, предпочитающая смотреть фильмы и сериалы про волшебство и магию

In [25]:
users = users.append({
    Columns.User: 10000003,
    'age': 'age_25_34',
    'income': 'income_40_60',
    'sex': 'Ж',
    'kids_flg': 0
}, ignore_index = True
)

In [26]:
magic_fan = items.loc[items['item_id'].isin(magic_fan.item_id), 
          ['item_id', 'title', 'content_type', 'countries', 'genre']]

In [27]:
magic_fan

,item_id,title,content_type,countries,genre
7,11114,Принцесса Лебедь: Пират или принцесса,film,США,"[для детей, сказки, полнометражные, зарубежные..."
19,15261,Спящая красавица. Легенда двух королевств: Лед...,film,Россия,"[русские, сказки, фильмы-спектакли, мюзиклы, с..."
20,2635,Рождество трех медведей,film,США,"[фильмы, для детей, сказки, зарубежные, семейное]"
29,6318,Днепровский ветер. Чары-камыши,film,СССР,"[драмы, русские, короткометражные]"
53,5780,Теория хаоса,film,США,"[драмы, мелодрамы, комедии]"


In [28]:
third_avatar = pd.DataFrame({
    'user_id': np.full(5, fill_value=10000003),
    'item_id': magic_fan.item_id,
    'last_watch_dt': np.full(5, fill_value='2021-07-25'),
    'total_dur': np.full(5, fill_value=np.nan),
    'watched_pct': [8.0, 100.0, 30.0, 5.0, 90.0],
    'weight': [1, 3, 3, 1, 3]
    }
)

third_avatar[Columns.Datetime] = pd.to_datetime(third_avatar[Columns.Datetime], format='%Y-%m-%d')

In [29]:
avatars = pd.concat((first_avatar, second_avatar, third_avatar))
train = train.append(avatars, ignore_index = True) 

In [30]:
avatars

,user_id,item_id,last_watch_dt,total_dur,watched_pct,weight
4,10000001,16268,2021-03-21,NaN,80.0,3
6,10000001,1468,2021-03-21,NaN,9.0,1
13,10000001,16429,2021-03-21,NaN,50.0,3
14,10000001,6181,2021-03-21,NaN,90.0,3
29,10000001,6318,2021-03-21,NaN,8.0,1
1,10000002,2508,2021-05-25,NaN,100.0,3
2,10000002,10716,2021-05-25,NaN,5.0,1
14,10000002,6181,2021-05-25,NaN,30.0,3
15,10000002,15076,2021-05-25,NaN,5.0,1
23,10000002,13109,2021-05-25,NaN,90.0,3


In [31]:
# соберем датасет
dataset = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type", "countries"],
)

avatars_ids = avatars['user_id'].unique()

In [32]:
# обучим модель на новых данных
model = LightFMWrapperModel(
    LightFM(
        no_components=246,
        learning_rate=0.020619, 
        loss='warp',
        rho=0.2,
        epsilon = 1e-5,
        user_alpha=0,
        item_alpha=0,
        random_state=42,
    ),
    epochs=1,
    num_threads=16,
)
model.fit(dataset)
recos = model.recommend(
    users=avatars_ids,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

In [33]:
recs = recos.merge(
    items[['item_id', 'title', 'content_type', 'countries', 'genre']], 
    on='item_id'
).sort_values(['user_id', 'rank'])

In [34]:
# мужчина средних лет, предпочитающий смотреть фильмы, которые выпустили в СССР
recs[recs['user_id'] == avatars_ids[0]]

,user_id,item_id,score,rank,title,content_type,countries,genre
0,10000001,13865,2.887706,1,Девятаев,film,Россия,"[драмы, военные, приключения]"
2,10000001,15297,2.615153,2,Клиника счастья,series,Россия,"[драмы, мелодрамы]"
4,10000001,10440,2.387885,3,Хрустальный,series,Россия,"[триллеры, детективы]"
5,10000001,7571,2.327745,4,100% волк,film,"Австралия, Бельгия","[мультфильм, приключения, семейное, фэнтези, к..."
7,10000001,12995,2.319194,5,Восемь сотен,film,Китай,"[боевики, драмы, историческое, военные]"
8,10000001,9728,2.297374,6,Гнев человеческий,film,"Великобритания, США","[боевики, триллеры]"
9,10000001,4740,2.249405,7,Сахаров. Две жизни,film,Россия,[документальное]
10,10000001,2499,2.201396,8,Падал прошлогодний снег,film,СССР,"[драмы, семейное, мультфильм, комедии]"
11,10000001,142,2.199917,9,Маша,film,Россия,"[драмы, триллеры]"
12,10000001,3734,2.143441,10,Прабабушка легкого поведения,film,Россия,[комедии]


В целом, рекомендации можно назвать релевантными, потому что в основном предлагается посмотреть фильмы, выпущенные уже в России

In [35]:
# молодая девушка, предпочитающая смотреть комедии
recs[recs['user_id'] == avatars_ids[1]]

,user_id,item_id,score,rank,title,content_type,countries,genre
14,10000002,3797,2.206792,1,[4k] Плохие парни навсегда,film,США,"[криминал, зарубежные, триллеры, боевики, коме..."
15,10000002,5125,2.138105,2,Фарго,film,США,"[криминал, драмы, зарубежные, триллеры, комедии]"
16,10000002,12523,2.126241,3,Очень опасная штучка,film,США,"[криминал, зарубежные, триллеры, мелодрамы, бо..."
17,10000002,7542,2.112513,4,Опасная иллюзия,film,США,"[криминал, драмы, зарубежные, триллеры, мелодр..."
18,10000002,9755,2.070572,5,Мачо и ботан 2,film,США,"[боевики, зарубежные, криминал, комедии]"
19,10000002,14804,2.043263,6,Криминальное чтиво,film,США,"[зарубежные, триллеры, криминал, комедии]"
20,10000002,3778,2.018189,7,Слепая ярость,film,США,"[криминал, драмы, зарубежные, триллеры, боевик..."
21,10000002,2719,2.016123,8,По признакам совместимости,film,США,"[мелодрамы, зарубежные, комедии]"
22,10000002,3120,1.998211,9,Вне времени,film,США,"[криминал, детективы, драмы, зарубежные, трилл..."
23,10000002,7686,1.997270,10,"Порнолоджи, или Милашка как ты",film,США,"[зарубежные, комедии, мелодрамы]"


Возможно, из-за генерации не совсем правильной выборки просмотренных фильмов получилось так, что пользователю предлагаются фильмы, которые совмещают в себе несколько жанров, один из которых - комедия

In [36]:
# молодая девушка, предпочитающая смотреть фильмы и сериалы про волшебство и магию
recs[recs['user_id'] == avatars_ids[2]]

,user_id,item_id,score,rank,title,content_type,countries,genre
3,10000003,15297,3.021473,1,Клиника счастья,series,Россия,"[драмы, мелодрамы]"
24,10000003,4151,2.860654,2,Секреты семейной жизни,series,Россия,[комедии]
13,10000003,3734,2.769945,3,Прабабушка легкого поведения,film,Россия,[комедии]
25,10000003,3182,2.767574,4,Ральф против Интернета,film,США,"[мультфильм, приключения, фантастика, семейное..."
6,10000003,7571,2.762947,5,100% волк,film,"Австралия, Бельгия","[мультфильм, приключения, семейное, фэнтези, к..."
26,10000003,2151,2.748451,6,[4k] Губка Боб в 3D,film,США,"[приключения, полнометражные, зарубежные, семе..."
27,10000003,15192,2.670235,7,[4k] Путеводная звезда,film,США,"[для детей, про животных, полнометражные, прик..."
28,10000003,16166,2.668572,8,Зверополис,film,США,"[приключения, мультфильм, детективы, комедии]"
29,10000003,13243,2.668092,9,Головоломка,film,США,"[фантастика, мультфильм, комедии]"
1,10000003,13865,2.642539,10,Девятаев,film,Россия,"[драмы, военные, приключения]"


Сложно объяснить почему первые позиции по скору занимают, кажется, совсем нерелевантные позиции, но остальные рекомендации отражают вкусы пользователя - в основном это фэнтэзийные фильмы, которые тоже в какой-то степени отражают тему волшебства и магии

## Холодные пользователи

Основной вариант - предлагать популярное за последнее время (неделя, две недели, месяц)

In [37]:
class Popular():
    def __init__(self, max_K=10, days=7, item_column='item_id', dt_column=Columns.Datetime):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, N=10):
        recs = self.recommendations[:N]
        return recs.tolist()

In [38]:
popular_model = Popular()
popular_model.fit(interactions)
popular_model.recommend()

[9728, 15297, 10440, 14488, 13865, 12192, 341, 4151, 3734, 512]

Таким образом пользователю будет предлагаться подборка популярного, если у него мало взаимодействий с айтемами